# Metrics on scikit-learn github repo

In [ ]:
import json
import datetime
import numpy as np
import pandas as pd

In [ ]:
with open("../data/issues.json") as f:
    istr = f.read()

In [ ]:
ijson = json.loads(istr)
ntot = len(ijson)

In [ ]:
creatDate = [ ijson[i]["node"]["createdAt"] for i in range(0, ntot) ]
closDate = [ ijson[i]["node"]["closedAt"] for i in range(0, ntot) ]
number = [ ijson[i]["node"]["number"] for i in range(0, ntot) ]
#author = [ ijson[i]["node"]["author"]["login"] for i in range(0, ntot) ]

In [ ]:
from datetime import datetime
from datetime import timedelta

crdate = []
cldate = []
duration = []
delta = timedelta(hours=1)

for i in range(0, ntot):
    crdate.append(pd.to_datetime(creatDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    cldate.append(pd.to_datetime(closDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    if closDate[i] != None:
        duration.append(pd.Timedelta.to_pytimedelta(cldate[i] - crdate[i]) / delta )
    else:
        duration.append(None)

In [ ]:
idata = { "Number" : number, "CreatedAt" : crdate, "ClosedAt" : cldate, "Duration" : duration }
idf = pd.DataFrame(idata)

In [ ]:
idf.head()

In [ ]:
import pandas_profiling
idf.profile_report()

In [ ]:
gidf = idf.groupby([idf['CreatedAt'].dt.to_period('Y')]).count().unstack()
gidf['CreatedAt'].plot(kind='bar', color='red')
gidf['ClosedAt'].plot(kind='bar', color='blue')